# Transformação de dados - Large Office

### Importando bibliotecas

In [2]:
import os
import pandas as pd


## Consolidando dados de consumo

In [4]:
# -----------------------------
# Caminhos das pastas
# -----------------------------
pasta_rateio = r"C:\Users\tbekho01.ATKEARNEY_AD\Kearney\Gran Coffee - Otimização de despesas operacionais - Project Management\5. Working Folder\24. Handover\250826_Otimização de Abastecedores_vEnviado\Dados Raw\Consumo\Rateio"
pasta_telemetria = r"C:\Users\tbekho01.ATKEARNEY_AD\Kearney\Gran Coffee - Otimização de despesas operacionais - Project Management\5. Working Folder\24. Handover\250826_Otimização de Abastecedores_vEnviado\Dados Raw\Consumo\Telemetria"
arquivo_gramaturas = r"C:\Users\tbekho01.ATKEARNEY_AD\Kearney\Gran Coffee - Otimização de despesas operacionais - Project Management\5. Working Folder\24. Handover\250826_Otimização de Abastecedores_vEnviado\Dados Raw\Gramaturas\Controle Gramaturas Oficial v_40.xlsx"

# -----------------------------
# Função para converter nome do arquivo em DATA
# Exemplo: "0725" -> 2025-07-01
# -----------------------------
def parse_nome_arquivo(nome_base):
    mes = int(nome_base[:2])
    ano = int(nome_base[2:])
    ano = 2000 + ano if ano < 100 else ano  # converte 25 -> 2025
    return pd.Timestamp(year=ano, month=mes, day=1)

# -----------------------------
# Carregar SKUs válidos do Excel de Gramaturas
# -----------------------------
df_gram = pd.read_excel(arquivo_gramaturas, sheet_name="Planilha1", skiprows=3, usecols=[0])
lista_skus_validos = df_gram.iloc[:303, 0].dropna().astype(str).unique().tolist()

# -----------------------------
# Processamento principal
# -----------------------------
arquivos_rateio = [f for f in os.listdir(pasta_rateio) if f.endswith(".xlsx")]

lista_dfs = []

for arquivo in arquivos_rateio:
    nome_base, ext = os.path.splitext(arquivo)
    caminho_rateio = os.path.join(pasta_rateio, arquivo)
    caminho_telemetria = os.path.join(pasta_telemetria, arquivo)
    
    # --- Ler arquivo de Rateio ---
    df_rateio = pd.read_excel(caminho_rateio)
    df_rateio = df_rateio[df_rateio["FATURADO"] == "SIM"]
    df_rateio = df_rateio[["CODPARC", "NOMEPARC", "CODBEM", "CODPROD", "PRODUTO", "CONSUMO"]].copy()
    df_rateio["DATA"] = parse_nome_arquivo(nome_base)
    
    # --- Ler arquivo de Telemetria correspondente ---
    if os.path.exists(caminho_telemetria):
        df_tel = pd.read_excel(caminho_telemetria)
        df_tel.columns = df_tel.columns.str.lower()
        
        # Filtrar linhas onde id_patrimonio não está em CODBEM do rateio
        df_tel = df_tel[~df_tel["id_patrimonio"].isin(df_rateio["CODBEM"])]
        
        # Selecionar colunas relevantes
        df_tel = df_tel[["id_patrimonio", "sku", "produto", "quantidade_total", "nomeparc"]]
        
        # Renomear colunas
        df_tel = df_tel.rename(columns={
            "id_patrimonio": "CODBEM",
            "sku": "CODPROD",
            "produto": "PRODUTO",
            "quantidade_total": "CONSUMO",
            "nomeparc": "NOMEPARC"
        })
        
        # 🔑 Filtrar apenas SKUs válidos
        df_tel = df_tel[df_tel["CODPROD"].astype(str).isin(lista_skus_validos)]
        
        # Garantir coluna CODPARC
        if "CODPARC" not in df_tel.columns:
            df_tel["CODPARC"] = None
        
        # Adicionar coluna DATA
        df_tel["DATA"] = df_rateio["DATA"].iloc[0] if not df_rateio.empty else pd.NaT
        
        # Unir os dois DataFrames
        df_unido = pd.concat([df_rateio, df_tel], ignore_index=True)
    else:
        df_unido = df_rateio
    
    lista_dfs.append(df_unido)

# -----------------------------
# Consolidação final
# -----------------------------
df_final = pd.concat(lista_dfs, ignore_index=True)

df_final["DATA"] = df_final["DATA"].dt.date

# Salvar resultado consolidado
saida = os.path.join(os.path.dirname(pasta_rateio), "Consumo_Consolidado.xlsx")
df_final.to_excel(saida, index=False)

print(f"✅ Processamento concluído! Arquivo salvo em: {saida}")


✅ Processamento concluído! Arquivo salvo em: C:\Users\tbekho01.ATKEARNEY_AD\Kearney\Gran Coffee - Otimização de despesas operacionais - Project Management\5. Working Folder\24. Handover\250826_Otimização de Abastecedores_vEnviado\Dados Raw\Consumo\Consumo_Consolidado.xlsx


## Capacidade de insumos por patrimônio

### Funções de padronização de string

In [ ]:
def std_codes(code):
    if str(code).replace('.','').isdigit():
        return(str(int(code))).replace('_x000D_\n', '').replace('\n', '')
    else:
        return str(code).replace('_x000D_\n', '').replace('\n', '')
    

def std_patr(code):
    if str(code).isdigit():
        return str(int(code))
    else:
        return str(code)

### Importando dados

In [ ]:
dados_path = #r"C:\Users\tbekho01\Kearney\Gran Coffee - Otimização de despesas operacionais - Project Management\5. Working Folder\15. Fase de Acompanhamento\Otimização de Abastecedores\Rollout\Rotas para ánalise_Rollout.xlsx"
lat_long_sp_path= #r"C:\Users\tbekho01\Kearney\Gran Coffee - Otimização de despesas operacionais - Project Management\5. Working Folder\15. Fase de Acompanhamento\Otimização de Abastecedores\Rollout\Endereços\SP\20250326_Lat_Long_SP.xlsx"

#Dados de consumo
patrimonios_sp = pd.read_excel(dados_path, sheet_name='Consumo Consolidado SP')

#Dados dos patrimonios
rotas_sp = pd.read_excel(dados_path, sheet_name='Rotas SP', skiprows=1)

#Latitude e Longitute dos parceiros
lat_long_sp = pd.read_excel(lat_long_sp_path, sheet_name='Sheet1')

#Capacidade das máquinas
capacidade_df = pd.read_excel(dados_path, sheet_name='Capacidade', skiprows = 1)

### Juntando os dados de consumo

In [ ]:
patrimonios_sp_clean=patrimonios_sp[["CODPARC", "NOMEPARC", "CODBEM", "CODPROD", "PRODUTO","CONSUMO DOSES", "Data"]]

patrimonios_sp_clean=patrimonios_sp_clean.rename(columns={"CONSUMO DOSES": "CONSUMO", "Data": "DATA"})

rotas_sp_clean=rotas_sp[["FILIAL", "PARCEIRO", "PATRIMÔNIO", "MODELO", "CLIENTE","TIPO DE MAQUINA", "CAPACIDADE EM DOSES"]]

lat_long_sp_clean=lat_long_sp[["PARCEIRO", "PATRIMÔNIO", "Latitude", "Longitude", "CLIENTE AJUSTADO ÚNICO"]]


### Aplicando padronização de strings

In [ ]:
patrimonios_sp_clean['NOMEPARC'] = patrimonios_sp_clean['NOMEPARC'].apply(std_codes)
patrimonios_sp_clean['CODBEM'] = patrimonios_sp_clean['CODBEM'].apply(std_codes)
patrimonios_sp_clean['CODBEM'] = patrimonios_sp_clean['CODBEM'].apply(std_patr)

rotas_sp_clean['PARCEIRO'] = rotas_sp_clean['PARCEIRO'].apply(std_codes)
rotas_sp_clean['PATRIMÔNIO'] = rotas_sp_clean['PATRIMÔNIO'].apply(std_codes)
rotas_sp_clean['PATRIMÔNIO'] = rotas_sp_clean['PATRIMÔNIO'].apply(std_patr)

lat_long_sp_clean['CLIENTE AJUSTADO ÚNICO'] = lat_long_sp_clean['CLIENTE AJUSTADO ÚNICO'].apply(std_codes)
lat_long_sp_clean['PATRIMÔNIO'] = lat_long_sp_clean['PATRIMÔNIO'].apply(std_codes)
lat_long_sp_clean['PATRIMÔNIO'] = lat_long_sp_clean['PATRIMÔNIO'].apply(std_patr)


### Junção dos dados

In [ ]:
patrimonios_completo = pd.merge(lat_long_sp_clean, rotas_sp_clean, left_on='PATRIMÔNIO', right_on='PATRIMÔNIO', how='inner')

patrimonios_completo=patrimonios_completo.drop(["PARCEIRO_x"], axis = 1)
patrimonios_completo=patrimonios_completo.rename(columns={"PARCEIRO_y": "PARCEIRO"})

capacidade_df_clean=capacidade_df.copy()
capacidade_df_clean = capacidade_df_clean[["MODELO", "NOMENCLATURA SIMPLIFICADA", "CAFÉ GRÃO", "LEITE", "CHOCOLATE", "CHÁ", "CAFÉ SOLÚVEL", "AÇÚCAR"]]

patrimonios_completo_capacidade = pd.merge(patrimonios_completo, capacidade_df, on='MODELO', how='inner')
patrimonios_completo_capacidade=patrimonios_completo_capacidade.rename(columns={"CAPACIDADE EM DOSES": "COPOS"})

### Preparando o arquivo

In [ ]:
# Lista de colunas de insumos
colunas_insumos = ['COPOS', 'CAFÉ GRÃO', 'LEITE', 'CHOCOLATE', 'CHÁ', 'CAFÉ SOLÚVEL', 'AÇÚCAR']

# Usando melt para transformar as colunas de insumos em uma única coluna 'INSUMO'
insumos = pd.melt(patrimonios_completo_capacidade, 
                  id_vars=['FILIAL', 'CLIENTE AJUSTADO ÚNICO', 'PATRIMÔNIO', 'MODELO'], 
                  value_vars=colunas_insumos,
                  var_name='INSUMO', 
                  value_name='CAPACIDADE')

# Renomeando a coluna 'CLIENTE AJUSTADO ÚNICO' para 'PARCEIRO'
insumos['PARCEIRO'] = insumos['CLIENTE AJUSTADO ÚNICO']

# Adicionando a coluna 'NIVEL DE REPOSIÇÃO' com o valor fixo de 0,3
insumos['NIVEL DE REPOSIÇÃO'] = 0.3

# Removendo a coluna 'CLIENTE AJUSTADO ÚNICO', pois já temos 'PARCEIRO'
insumos = insumos.drop(columns=['CLIENTE AJUSTADO ÚNICO'])

# Convertendo a coluna 'CAPACIDADE' para numérico, forçando erros para NaN
insumos['CAPACIDADE'] = pd.to_numeric(insumos['CAPACIDADE'], errors='coerce')

# Removendo as linhas onde 'CAPACIDADE' não é numérica (NaN)
insumos = insumos.dropna(subset=['CAPACIDADE'])

# Reorganizando as colunas na ordem desejada
insumos = insumos[['FILIAL', 'PARCEIRO', 'PATRIMÔNIO', 'INSUMO', 'CAPACIDADE', 'NIVEL DE REPOSIÇÃO', 'MODELO']]

# Exibindo o resultado
insumos

### Exportando a aba "insumos" de Dados.xlsx

In [ ]:
# Caminho para o novo arquivo de saída
output_path = 

# Exportando o DataFrame 'insumos' para o novo arquivo Excel
insumos.to_excel(output_path, index=False, sheet_name="insumos")

# Mensagem de confirmação
print(f"Arquivo exportado com sucesso para: {output_path}")


## Consumo de insumos por patrimônio

### Importando dados

In [ ]:
#Dado de gramaturas por doses
gramaturas_sku = #pd.read_excel(dados_path, sheet_name='Gramaturas_concat_ROLLOUT', skiprows=2, usecols="B:R")


In [ ]:
# Convert DATA column to datetime
patrimonios_sp_clean['DATA'] = pd.to_datetime(patrimonios_sp_clean['DATA'])

# Create INICIO (first day of the month)
patrimonios_sp_clean['INICIO'] = patrimonios_sp_clean['DATA'].dt.to_period('M').dt.start_time

# Create FIM (last day of the month)
patrimonios_sp_clean['FIM'] = patrimonios_sp_clean['DATA'].dt.to_period('M').dt.end_time

# Drop the original DATA column
patrimonios_sp_clean = patrimonios_sp_clean.drop(columns=['DATA'])

# Format INICIO and FIM to show only DD/MM/YYYY
patrimonios_sp_clean['INICIO'] = patrimonios_sp_clean['INICIO'].dt.strftime('%d/%m/%Y')
patrimonios_sp_clean['FIM'] = patrimonios_sp_clean['FIM'].dt.strftime('%d/%m/%Y')

# Display result
patrimonios_sp_clean

In [ ]:
merged_df = pd.merge(patrimonios_sp_clean, gramaturas_sku,
                         left_on='CODPROD', right_on='SKU', how='inner')

merged_df = merged_df.dropna(subset=['DOSE FINAL (mL)']).drop(["DOSE FINAL (mL)", "PALHETA", "Produto", "Obs", "Considerar como copo", "SKU"], axis =1)

insumos2 = ['CAFÉ GRÃO', 'LEITE', 'CHOCOLATE', 'CHÁ', 'CAFÉ SOLÚVEL', 'AÇÚCAR', "CAFÉ COM LEITE CARAMELO", "CAPPUCCINO COM CANELA CAFÉ DO CENTRO", "CAPPUCCINO SEM CANELA CAFÉ DO CENTRO", "COPO"]
for insumos in insumos2:
    merged_df[insumos] *= merged_df['CONSUMO']


In [ ]:
merged_df

In [ ]:
# Somar as colunas especificadas para esses CODBEMs
soma_por_codbem = merged_df.groupby(['CODPARC','CODBEM', 'INICIO', 'FIM'])[['CAFÉ GRÃO', 'LEITE', 'CHOCOLATE', 'CHÁ', 'CAFÉ SOLÚVEL', 'AÇÚCAR', 'CAFÉ COM LEITE CARAMELO', 'CAPPUCCINO COM CANELA CAFÉ DO CENTRO', 'CAPPUCCINO SEM CANELA CAFÉ DO CENTRO', 'COPO']].sum().reset_index()

In [ ]:
melted_df = pd.melt(soma_por_codbem, 
                    id_vars=['CODPARC', 'CODBEM', 'INICIO', 'FIM'],  # Colunas que ficam fixas
                    value_vars=['CAFÉ GRÃO', 'LEITE', 'CHOCOLATE', 'CHÁ', 'CAFÉ SOLÚVEL', 
                                 'AÇÚCAR', 'CAFÉ COM LEITE CARAMELO', 
                                 'CAPPUCCINO COM CANELA CAFÉ DO CENTRO',
                                 'CAPPUCCINO SEM CANELA CAFÉ DO CENTRO', 'COPO'],  # Colunas que vão virar INSUMO
                    var_name='INSUMO',  # Nome da nova coluna para as variáveis de ingredientes
                    value_name='CONSUMO')  # Nome da nova coluna para os valores

# Renomeando as colunas conforme a estrutura desejada
melted_df['FILIAL'] = "SP" # A coluna FILIAL será o CODPARC
melted_df['PARCEIRO'] = melted_df['CODPARC']  # A coluna PARCEIRO será o CODBEM
melted_df['PATRIMONIO'] = melted_df['CODBEM']  # PATRIMONIO também será o CODBEM (pode ser ajustado conforme a lógica)

# Selecionando e reorganizando as colunas conforme a ordem desejada
final_df = melted_df[['FILIAL', 'PARCEIRO', 'PATRIMONIO', 'INSUMO', 'CONSUMO', 'INICIO', 'FIM']]

# Exibindo as primeiras linhas do DataFrame final
final_df = final_df.sort_values(by=['FILIAL', 'PARCEIRO', 'PATRIMONIO',"INICIO", "FIM", 'INSUMO'])
final_df

### Criando um ranking dos compartimentos de maior capacidade por patrimônio (para alocar os insumos especiais)

In [ ]:



# 1. Filtrando os insumos desejados (CHÁ, CHOCOLATE, CAFÉ SOLÚVEL)
insumos_filtrados = insumos[insumos['INSUMO'].isin(['CHÁ', 'CHOCOLATE', 'CAFÉ SOLÚVEL'])]

# 2. Agrupando por PARCEIRO e PATRIMÔNIO, e ordenando por CAPACIDADE dentro de cada grupo
# Vamos usar sort_values para ordenar por CAPACIDADE
def ranking_insumos(group):
    # Classificando por CAPACIDADE em ordem decrescente (do maior para o menor)
    group_sorted = group.sort_values(by='CAPACIDADE', ascending=False)

    # Criando o ranking de 1 a 3 para os insumos dentro do grupo
    group_sorted['RANK'] = group_sorted['CAPACIDADE'].rank(ascending=False, method='first')

    # Pegando as colunas de RANK 1, 2, 3
    rank_1 = group_sorted[group_sorted['RANK'] == 1]['INSUMO'].values[0] if len(group_sorted[group_sorted['RANK'] == 1]) > 0 else None
    rank_2 = group_sorted[group_sorted['RANK'] == 2]['INSUMO'].values[0] if len(group_sorted[group_sorted['RANK'] == 2]) > 0 else None
    rank_3 = group_sorted[group_sorted['RANK'] == 3]['INSUMO'].values[0] if len(group_sorted[group_sorted['RANK'] == 3]) > 0 else None

    # Retornando a linha com o PARCEIRO, PATRIMÔNIO e os 3 insumos no ranking
    return pd.Series({
        'PARCEIRO': group['PARCEIRO'].values[0],
        'PATRIMÔNIO': group['PATRIMÔNIO'].values[0],
        'CAPACIDADE_1': rank_1,
        'CAPACIDADE_2': rank_2,
        'CAPACIDADE_3': rank_3
    })

# 3. Aplicando a função para cada grupo de PARCEIRO e PATRIMÔNIO
ranked_insumos = insumos_filtrados.groupby(['PARCEIRO', 'PATRIMÔNIO']).apply(ranking_insumos)

# 4. Resetando o índice do DataFrame final para uma estrutura mais limpa
ranked_insumos = ranked_insumos.reset_index(drop=True)

# Mostrando o DataFrame final com os rankings
ranked_insumos

### Alocando os insumos especiais nos compartimentos vazios dos patrimônios que possuem estes insumos

In [ ]:

final_df_aux = final_df.copy()

# Supondo que final_df_aux e ranked_insumos já estão definidos
# final_df_aux é o dataframe principal, ranked_insumos é o dataframe com o ranking dos insumos

# Lista de insumos a serem verificados
target_insumos = ['CAFÉ COM LEITE CARAMELO', 
                  'CAPPUCCINO COM CANELA CAFÉ DO CENTRO',
                  'CAPPUCCINO SEM CANELA CAFÉ DO CENTRO']

# Passo 1: Filtrar as linhas onde o INSUMO é um dos alvos e o CONSUMO não é 0
mask_target_insumos = final_df_aux['INSUMO'].isin(target_insumos)
target_rows = final_df_aux[mask_target_insumos]

# Passo 2: Processar cada linha
rows_to_drop = []  # Para armazenar os índices das linhas a serem removidas

for idx, row in target_rows.iterrows():
    # Se CONSUMO for 0, ignorar e remover a linha
    if row['CONSUMO'] == 0:
        rows_to_drop.append(idx)
        continue

    # Passo 3: Verificar se CHÁ, CHOCOLATE ou CAFÉ SOLÚVEL têm CONSUMO igual a zero
    zero_consumption_ingredients = []
    
    for ingredient in ['CHÁ', 'CHOCOLATE', 'CAFÉ SOLÚVEL']:
        ingredient_row = final_df_aux[(final_df_aux['INSUMO'] == ingredient) & 
                                  #(final_df_aux['FILIAL'] == row['FILIAL']) & 
                                  (final_df_aux['PARCEIRO'] == row['PARCEIRO']) & 
                                  (final_df_aux['PATRIMONIO'] == row['PATRIMONIO']) & 
                                  (final_df_aux['INICIO'] == row['INICIO']) & 
                                  (final_df_aux['FIM'] == row['FIM'])]
        
        # Verificar se o CONSUMO é igual a zero e adicionar ao lista de insumos com consumo zero
        if not ingredient_row.empty and ingredient_row['CONSUMO'].values[0] == 0:
            zero_consumption_ingredients.append(ingredient)
    
    # Se mais de um insumo tiver CONSUMO igual a zero
    if len(zero_consumption_ingredients) > 0:
        # Passo 4: Verificar qual insumo tem o maior ranking de capacidade no ranked_insumos
        print(zero_consumption_ingredients)
        print(row['PATRIMONIO'])
        
        if row['PATRIMONIO'] in ranked_insumos["PATRIMÔNIO"].unique():
            print("s")
            insumo_cap_1 = ranked_insumos[ranked_insumos['PATRIMÔNIO'] == row['PATRIMONIO']]['CAPACIDADE_1'].values[0]
            insumo_cap_2 = ranked_insumos[ranked_insumos['PATRIMÔNIO'] == row['PATRIMONIO']]['CAPACIDADE_2'].values[0]
            insumo_cap_3 = ranked_insumos[ranked_insumos['PATRIMÔNIO'] == row['PATRIMONIO']]['CAPACIDADE_3'].values[0]
            
            if insumo_cap_1 in zero_consumption_ingredients:
                target_ingredient = insumo_cap_1 
            elif insumo_cap_2 in zero_consumption_ingredients:
                target_ingredient = insumo_cap_2
            elif insumo_cap_3 in zero_consumption_ingredients:
                target_ingredient = insumo_cap_3 

            ingredient_row = final_df_aux[(final_df_aux['INSUMO'] == target_ingredient) & 
                                #(final_df_aux['FILIAL'] == row['FILIAL']) & 
                                (final_df_aux['PARCEIRO'] == row['PARCEIRO']) & 
                                (final_df_aux['PATRIMONIO'] == row['PATRIMONIO']) & 
                                (final_df_aux['INICIO'] == row['INICIO']) & 
                                (final_df_aux['FIM'] == row['FIM'])]
            
            # Transferir o valor de CONSUMO para o insumo com maior ranking
            final_df_aux.loc[ingredient_row.index, 'CONSUMO'] = row['CONSUMO']
            final_df_aux.loc[ingredient_row.index, 'TRANSFER'] = row["INSUMO"]
            
            # Após a transferência, marcar a linha original para exclusão
            rows_to_drop.append(idx)
        else:
            print("n")
            rows_to_drop.append(idx)

# Passo 5: Deletar as linhas marcadas para exclusão
final_df_aux = final_df_aux.drop(rows_to_drop)

# Resetar o índice
final_df_aux = final_df_aux.reset_index(drop=True)

# Mostrar o DataFrame final
final_df_aux

In [ ]:
#Retirando as linhas em que o consumo é 0
final_df_aux_export= final_df_aux[final_df_aux["CONSUMO"] != 0]

### Exportando Dados de Consumo.csv

In [ ]:

# Caminho de saída
output_path = #, 'Dados de Consumo.csv')

# Exportar para CSV com encoding UTF-8
final_df_aux_export.to_csv(output_path, index=False, encoding='cp1252', sep=';')

# Mensagem de confirmação
print(f"Arquivo consolidado exportado com sucesso para: {output_path}")